# Pancake Pridiction v2

In [492]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import datetime as dt
import pandas as pd
import numpy as n
import math

pd.set_option('display.max_rows', None)

In [493]:
import os
import sys

current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
sys.path.append(project_dir) if project_dir not in sys.path else None

import pancake
psp = pancake.Prediction()

### Config

In [494]:
dataset_reconds = 300

### Fetching Data

In [495]:
round_columns = ["epoch",
                 "startTimestamp",
                 "lockTimestamp",
                 "closeTimestamp",
                 "lockPrice",
                 "closePrice",
                 "lockOracleId",
                 "closeOracleId",
                 "totalAmount",
                 "bullAmount",
                 "bearAmount",
                 "rewardBaseCalAmount",
                 "rewardAmount",
                 "oracleCalled"]

def get_history(_psp, current_epoch, back_in_time=100):
    start_epoch_history = current_epoch - 2 - back_in_time

    df_history_round = pd.DataFrame(columns=round_columns)
    for i in range(start_epoch_history, current_epoch - 1):
        df_round = _psp.get_round(i)
        df_history_round = df_history_round.append(df_round)

    df_history_round = df_history_round.sort_values('epoch', ascending=False)
    df_history_round = df_history_round.reset_index(drop=True)
    
    return df_history_round

#### Creating a dataset

In [496]:
current_epoch = psp.get_current_epoch()
df = get_history(psp, current_epoch, dataset_reconds)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled
0,38395.0,1.642776e+09,1.642776e+09,1.642776e+09,420.025513,418.568878,0.0,0.0,46.665051,13.046211,33.618839,33.618839,45.265099,True
1,38394.0,1.642775e+09,1.642776e+09,1.642776e+09,419.410004,420.025513,0.0,0.0,27.337952,13.910913,13.427039,13.910913,26.517813,True
2,38393.0,1.642775e+09,1.642775e+09,1.642776e+09,417.799988,419.410004,0.0,0.0,47.518555,22.770111,24.748446,22.770111,46.092999,True
3,38392.0,1.642775e+09,1.642775e+09,1.642775e+09,416.010010,417.799988,0.0,0.0,31.646482,16.855942,14.790541,16.855942,30.697088,True
4,38391.0,1.642774e+09,1.642775e+09,1.642775e+09,415.905884,416.010010,0.0,0.0,42.517879,19.623856,22.894024,19.623856,41.242344,True


### Adding result

In [497]:
def add_result(lockPrice, closePrice):
    if lockPrice == closePrice:
        # draw
        return 0
    elif lockPrice > closePrice:
        # bear
        return -1
    elif lockPrice < closePrice:
        # bull
        return 1
        
df["result"] = df.apply(lambda row : add_result(row["lockPrice"], row["closePrice"]), axis=1)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled,result
0,38395.0,1.642776e+09,1.642776e+09,1.642776e+09,420.025513,418.568878,0.0,0.0,46.665051,13.046211,33.618839,33.618839,45.265099,True,-1
1,38394.0,1.642775e+09,1.642776e+09,1.642776e+09,419.410004,420.025513,0.0,0.0,27.337952,13.910913,13.427039,13.910913,26.517813,True,1
2,38393.0,1.642775e+09,1.642775e+09,1.642776e+09,417.799988,419.410004,0.0,0.0,47.518555,22.770111,24.748446,22.770111,46.092999,True,1
3,38392.0,1.642775e+09,1.642775e+09,1.642775e+09,416.010010,417.799988,0.0,0.0,31.646482,16.855942,14.790541,16.855942,30.697088,True,1
4,38391.0,1.642774e+09,1.642775e+09,1.642775e+09,415.905884,416.010010,0.0,0.0,42.517879,19.623856,22.894024,19.623856,41.242344,True,1


### Maximum Consequitive Occurance

In [498]:
countDf = pd.DataFrame({'result': list(df['result'][(df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(df['result'].groupby((df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

bear_count = countDf[countDf.result == -1]['runs'].max()
bull_count = countDf[countDf.result == 1]['runs'].max()
draw_count = countDf[countDf.result == 0]['runs'].max()

print(f"Bear {bear_count} / Bull {bull_count} / Draw {draw_count}")

Bear 8 / Bull 9 / Draw nan


## Strategies

In [499]:
running_columns = ["epoch", "position", "amount", "reward", "recent_loss", "factor"]
df_running = pd.DataFrame(columns=running_columns)

In [500]:
def bet(df_running, epoch, position, amount, recent_loss, factor):
    data = [epoch, position, amount, 0, recent_loss, factor]
    temp = pd.DataFrame(data=[data], columns=running_columns)
    df_running = df_running.append(temp)
    return df_running

    
def get_round_stats(df, epoch):
    df_round = df[df.epoch == epoch]

    total_amount = (df_round["bullAmount"] + df_round["bearAmount"]).iloc[0]
    if total_amount > 0:
        bull_ratio = ((df_round["bullAmount"] / total_amount) * 100).iloc[0]
        bear_ratio = ((df_round["bearAmount"] / total_amount) * 100).iloc[0]
        bear_pay_ratio = (total_amount / df_round["bearAmount"]).iloc[0]
        bull_pay_ratio = (total_amount / df_round["bullAmount"]).iloc[0]
    else:
        bull_ratio = None
        bear_ratio = None
        bear_pay_ratio = None
        bull_pay_ratio = None

    lockPrice = df_round["lockPrice"].iloc[0]
    closePrice = df_round["closePrice"].iloc[0]

    if lockPrice == closePrice:
        # draw
        result = 0
    elif lockPrice > closePrice:
        # bear
        result = -1
    elif lockPrice < closePrice:
        # bull
        result = 1
    
    return {"total_amount": total_amount,
           "bull_ratio": bull_ratio, "bear_ratio": bear_ratio,
           "bear_pay_ratio": bear_pay_ratio, "bull_pay_ratio": bull_pay_ratio,
           "result": result}


def update_reward(df, df_running, epoch, status):
    bet_value = df_running[df_running.epoch == epoch]["amount"].iloc[0]
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    
    if status == 0:
        df_running.loc[df_running.epoch == epoch, "reward"] = -1 * bet_value
        pay_ratio = 0
    elif status == 1:
        epoch_stats = get_round_stats(df, epoch)
        if bet_position == "bull":
            pay_ratio = epoch_stats["bull_pay_ratio"]
        elif bet_position == "bear":
            pay_ratio = epoch_stats["bear_pay_ratio"]

        df_running.loc[df_running.epoch == epoch, "reward"] = (bet_value * pay_ratio) - bet_value
    
    print(f"[{epoch}] - Factor {pay_ratio} - Bet {bet_value} - Position {position}")    
    return df_running


def check_result(df, df_running, epoch):
    
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    epoch_stats = get_round_stats(df, epoch)
    
    if bet_position == "bull" and epoch_stats["result"] == 1:
        result = 1
    elif bet_position == "bear" and epoch_stats["result"] == -1:
        result = 1
    else:
        result = 0
    
    df_running = update_reward(df, df_running, epoch, result)
    return df_running, epoch_stats["result"]
    

def overview(df_running):
    total_spent = df_running.sum()["amount"]
    total_loss = abs(df_running[df_running["reward"] < 0].sum()["reward"])
    loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    estimated_win = df_running[df_running["reward"] > 0].sum()["reward"]
    win_times = df_running[df_running["reward"] > 0].count()["reward"]
    estimated_gain = df_running.sum()["reward"]
    max_spent = df_running.max()["amount"]

    last_win_epoch = df_running[df_running["reward"] > 0].max()["epoch"]
    if last_win_epoch is None or math.isnan(last_win_epoch):
        recent_loss = abs(df_running.sum()["reward"])
        recent_loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    else:
        recent_loss = abs(df_running[df_running["epoch"] > last_win_epoch].sum()["reward"])
        recent_loss_times = df_running[(df_running["epoch"] > last_win_epoch)
                                       & (df_running["reward"] < 0)].count()["reward"]

    return {"total_spent": total_spent,
            "max_spent": max_spent,
            "total_loss": total_loss,
            "loss_times": loss_times,
            "estimated_win": estimated_win,
            "win_times": win_times,
            "estimated_gain": estimated_gain,
            "recent_loss": recent_loss,
            "recent_loss_times": recent_loss_times}


### Strategy 1: Trend

### Strategy 2: Same-Before

### Strategy 3: Random

In [501]:
import random

starting_epoch = df.min()["epoch"] + 20
maximum_epoch = df.max()["epoch"] - 1

# 0: Even / 1: Odd
strategy_remainder = 0
base_bet = 0.001
safe_bet = 0.2

current_epoch = starting_epoch
value = base_bet
print("Starting Epoch: ", starting_epoch)

Starting Epoch:  38115.0


In [502]:
while current_epoch < maximum_epoch:
    if current_epoch % 2 == strategy_remainder:
    
        bet_status = overview(df_running)
        current_round_stats = get_round_stats(df, current_epoch)

        rand = random.getrandbits(1)

        if rand:
            # bull
            custom_factor = current_round_stats["bull_pay_ratio"] - safe_bet
            position = "bull"
        else:
            # bear
            custom_factor = current_round_stats["bear_pay_ratio"] - safe_bet
            position = "bear"

        value = (bet_status["recent_loss"] + base_bet) / (custom_factor - 1)
        if value < base_bet:
            value = base_bet

        df_running = bet(df_running, current_epoch, position, value, bet_status["recent_loss"], custom_factor)
        df_running, result = check_result(df, df_running, current_epoch)
        print(f"[{current_epoch}] - {position} - {value}")
    current_epoch += 1

[38116.0] - Factor 0 - Bet 0.001 - Position bear
[38116.0] - bear - 0.001
[38118.0] - Factor 1.5872827768325806 - Bet 0.005164185240451797 - Position bull
[38118.0] - bull - 0.005164185240451797
[38120.0] - Factor 1.5501972436904907 - Bet 0.0028555336114632988 - Position bull
[38120.0] - bull - 0.0028555336114632988
[38122.0] - Factor 1.9363503456115723 - Bet 0.0013580492030182381 - Position bear
[38122.0] - bear - 0.0013580492030182381
[38124.0] - Factor 0 - Bet 0.0010758553340575977 - Position bear
[38124.0] - bear - 0.0010758553340575977
[38126.0] - Factor 0 - Bet 0.001 - Position bull
[38126.0] - bull - 0.001
[38128.0] - Factor 0 - Bet 0.003481037110399529 - Position bull
[38128.0] - bull - 0.003481037110399529
[38130.0] - Factor 1.7516357898712158 - Bet 0.011886270914343484 - Position bull
[38130.0] - bull - 0.011886270914343484
[38132.0] - Factor 0 - Bet 0.001 - Position bull
[38132.0] - bull - 0.001
[38134.0] - Factor 0 - Bet 0.003834241500032315 - Position bear
[38134.0] - bear

### Overview

In [503]:
sum_reward = 0
df_running = df_running.reset_index(drop=True)

for index, row in df_running.iterrows():
        reward = row["reward"]
        sum_reward += reward
        
        df_running.loc[df_running.index == index, "acc_reward"] = sum_reward
        
df_running

,epoch,position,amount,reward,recent_loss,factor,acc_reward
0,38116.0,bear,0.001000,-0.001,0.000000,2.084882,-0.001000
1,38118.0,bull,0.005164,0.003033,0.001000,1.387283,0.002033
2,38120.0,bull,0.002856,0.001571,0.000000,1.350197,0.003604
3,38122.0,bear,0.001358,0.001272,0.000000,1.736350,0.004876
4,38124.0,bear,0.001076,-0.001076,0.000000,1.929493,0.003800
5,38126.0,bull,0.001000,-0.001,0.001076,3.508560,0.002800
6,38128.0,bull,0.003481,-0.003481,0.002076,1.883603,-0.000681
7,38130.0,bull,0.011886,0.008934,0.005557,1.551636,0.008253
8,38132.0,bull,0.001000,-0.001,0.000000,2.261254,0.007253
9,38134.0,bear,0.003834,-0.003834,0.001000,1.521616,0.003419


In [504]:
summary = overview(df_running)
print(summary)

{'total_spent': 0.8357400197875616, 'max_spent': 0.10489134746729396, 'total_loss': 0.2596882730340293, 'loss_times': 67, 'estimated_win': 0.44978900496072466, 'win_times': 72, 'estimated_gain': 0.19010073192669522, 'recent_loss': 0.0, 'recent_loss_times': 0}


### Counting Consecutive Win/Loss

In [505]:
def add_result_binary(reward):
    if reward > 0:
        # bull
        return 1
    else:
        # bear
        return -1
        
temp_df = df_running.copy()

temp_df["result"] = temp_df.apply(lambda row : add_result_binary(row["reward"]), axis=1)

countDf = pd.DataFrame({'result': list(temp_df['result'][(temp_df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(temp_df['result'].groupby((temp_df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

loss_count = countDf[countDf.result == -1]['runs'].max()
win_count = countDf[countDf.result == 1]['runs'].max()

print(f"Loss {loss_count} / Win {win_count}")

Loss 5 / Win 8
